In [ ]:
import numpy as np
from scipy import constants
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import asyncio
import plotly.io as pio
# pio.templates.default = "seaborn"


In [ ]:
class CanonicalMonteCarloSimulation:
    def __init__(self, L):
        self.L = L # Lattice size
        self.c_B = 0.5 # Concentration of B atoms
        self.initialize_lattice()

    def initialize_lattice(self):
        num_B_atoms = int(self.L**2 * self.c_B)
        num_A_atoms = self.L**2 - num_B_atoms
        # A L by L array representing the lattice using +1 for A and -1 for B
        lattice_values = np.array([-1] * num_B_atoms + [1] * num_A_atoms)
        np.random.shuffle(lattice_values)
        self.lattice = lattice_values.reshape((self.L, self.L))

    def initialize_parameters(self, T, E_AA, E_BB, E_AB, c_B):
        self.T = T # Temperature
        self.E_AA = E_AA # AA bond energy
        self.E_BB = E_BB # BB bond energy
        self.E_AB = E_AB # AB bond energy
        self.c_B = c_B # Concentration of B atoms
        self.initialize_lattice()
        # Total energy per atom
        self.total_energy_per_atom = self.get_total_energy()/(self.L**2) 

    # Helper function to calculate energy contribution for a atom and its neighbors
    def calculate_energy_contribution(self, i, j):
        atom_type = self.lattice[i, j]
        # Neighbors in 4 directions with periodic boundary conditions
        neighbor_position_lists = [((i+1)%self.L, j), ((i-1)%self.L, j), (i, (j+1)%self.L), (i, (j-1)%self.L)]
        E = 0
        for ni, nj in neighbor_position_lists:
            neighbor_type = self.lattice[ni, nj]
            if atom_type == 1 and neighbor_type == 1:
                E += self.E_AA
            elif atom_type == -1 and neighbor_type == -1:
                E += self.E_BB
            else:
                E += self.E_AB
        return E

    # Define a function to calculate the total energy using bond counting model
    def get_total_energy(self):
        E = 0
        for i in range(self.L):
            for j in range(self.L):
                E += self.calculate_energy_contribution(i, j)
        return E/2 # Each bond is counted twice

    # Define a function to calculate the energy change if two atoms are swapped
    # The positions of the two atoms are pos1 = (i1, j1) and pos2 = (i2, j2)
    def get_energy_change(self, pos1, pos2):
        i1, j1 = pos1
        i2, j2 = pos2
        atom1_type, atom2_type = self.lattice[i1, j1], self.lattice[i2, j2]
        # Ignore if the same type of atoms are picked
        if (i1 == i2 and j1 == j2) or (atom1_type == atom2_type):
            return 0
        dE = 0
        if (i2 in [(i1+1)%self.L, i1 ,(i1-1)%self.L] and j2 in [(j1+1)%self.L, j1 ,(j1-1)%self.L]):
            # The two atoms are nearest or second-nearest neighbors
            E_before = self.get_total_energy()
            self.lattice[i1, j1], self.lattice[i2, j2] = self.lattice[i2, j2], self.lattice[i1, j1]
            E_after = self.get_total_energy()
            self.lattice[i1, j1], self.lattice[i2, j2] = self.lattice[i2, j2], self.lattice[i1, j1]
            dE = E_after - E_before
        else:
            dE_before = self.calculate_energy_contribution(i1, j1) + \
                        self.calculate_energy_contribution(i2, j2)
            self.lattice[i1, j1], self.lattice[i2, j2] = self.lattice[i2, j2], self.lattice[i1, j1]
            dE_after = self.calculate_energy_contribution(i1, j1) + \
                    self.calculate_energy_contribution(i2, j2)
            self.lattice[i1, j1], self.lattice[i2, j2] = self.lattice[i2, j2], self.lattice[i1, j1]
            dE = dE_after - dE_before
        return dE
   
    # Define a function to attempt to swap two atoms and update the energy 
    def one_simulation_step(self):
        # Pick two random atoms in the form of pos1 = (i1, j1) and pos2 = (i2, j2)
        kB = constants.value('Boltzmann constant in eV/K')
        i1, j1 = np.random.randint(0, self.L, size=2)
        i2, j2 = np.random.randint(0, self.L, size=2)
        dE = self.get_energy_change((i1, j1), (i2, j2))
        if dE < 0 or np.random.rand() < np.exp(-dE / (kB * self.T)):
            # Successful swap
            self.lattice[i1, j1], self.lattice[i2, j2] = self.lattice[i2, j2], self.lattice[i1, j1]
            self.total_energy_per_atom += dE / (self.L**2)

    def calculate_warren_cowley_sro(self):
        # Calculate the Warren-Cowley short-range order parameter
        # The order parameter is defined as alpha_AB = 1 - (P_AB)/(2*c_A*c_B)
        n_total = self.L*self.L*2 # Total number of bonds
        n_AB = 0
        for i in range(self.L):
            for j in range(self.L):
                atom_type = self.lattice[i, j]
                # Neighbors in 4 directions with periodic boundary conditions
                neighbor_position_lists = [((i+1)%self.L, j), ((i-1)%self.L, j),
                                           (i, (j+1)%self.L), (i, (j-1)%self.L)]
                for ni, nj in neighbor_position_lists:
                    if atom_type != self.lattice[ni, nj]:
                        n_AB += 1
        p_AB = n_AB/n_total/2
        return 1 - p_AB / (2 * self.c_B * (1 - self.c_B))

In [ ]:
fig = go.FigureWidget(make_subplots(rows=2, cols=2, specs=[[{"rowspan": 2}, {}], [None, {}]]))
fig.update_layout(height=600, width=1200)

def initialize_plot(lattice):
    type_A_x, type_A_y = np.where(lattice == 1)
    type_B_x, type_B_y = np.where(lattice == -1)
    scatterA = go.Scatter(x=type_A_x + 0.5, y=type_A_y + 0.5, mode='markers', marker_line_width=.1,
                          marker=dict(color='#FF9D3B', size=12), name='Atom A')
    scatterB = go.Scatter(x=type_B_x + 0.5, y=type_B_y + 0.5, mode='markers', marker_line_width=.1,
                          marker=dict(color='#37BCFF', size=12), name='Atom B')
    fig.add_trace(scatterA, row=1, col=1)
    fig.add_trace(scatterB, row=1, col=1)
    fig.update_xaxes(range=[0, L], row=1, col=1)
    fig.update_yaxes(range=[0, L], row=1, col=1)
    energy_plot = go.Scatter(x=[], y=[], mode='lines+markers', name='Energy')
    fig.add_trace(energy_plot, row=1, col=2)
    fig.update_xaxes(title_text="Steps", row=1, col=2)
    fig.update_yaxes(title_text="Energy (eV/atom)", row=1, col=2)
    sro_plot = go.Scatter(x=[], y=[], mode='lines+markers', name='SRO')
    fig.add_trace(sro_plot, row=2, col=2)
    fig.update_xaxes(title_text="Steps", row=2, col=2)
    fig.update_yaxes(title_text="Warren-Cowley parameter A-B bond", row=2, col=2)

def updater_plot(lattice, step_list, energy_list, sro_list):
    type_A_x, type_A_y = np.where(lattice == 1)
    type_B_x, type_B_y = np.where(lattice == -1)
    with fig.batch_update():
        fig.data[0].x = type_A_x + 0.5
        fig.data[0].y = type_A_y + 0.5
        fig.data[1].x = type_B_x + 0.5
        fig.data[1].y = type_B_y + 0.5
        fig.data[2].x = step_list
        fig.data[2].y = energy_list
        fig.data[3].x = step_list
        fig.data[3].y = sro_list

# Define the animation function
async def animate_simulation(steps, T, interval, E_AA, E_BB, E_AB, C_B):
    global stop_animation
    MC.initialize_parameters(T, E_AA, E_BB, E_AB, C_B)
    step_list = []
    energy_list = []
    sro_list = []
    for step in range(steps+1):
        if stop_animation:
            break
        MC.one_simulation_step()
        if step % interval == 0:
            step_list.append(step)
            energy_list.append(MC.total_energy_per_atom)
            sro = MC.calculate_warren_cowley_sro()
            sro_list.append(sro)
            updater_plot(MC.lattice, step_list, energy_list, sro_list)
            step_info.value = f"Current Step: {step}, Energy: {MC.total_energy_per_atom:.4f} eV/atom, Warren-Cowley A-B bond: {sro:.4f}"
        await asyncio.sleep(1/interval/3)  # Allow interruption

# Callback function for play button
async def on_play_button_clicked(b):
    global stop_animation
    stop_animation = False
    play_button.disabled = True  # Disable the play button
    await animate_simulation(int(total_steps_input.value), float(temperature_input.value),
                            int(interval_input.value), float(E_AA_input.value),
                            float(E_BB_input.value), float(E_AB_input.value),
                            float(C_B_slider.value) / 100)

# Callback function for stop button
def on_stop_button_clicked(b):
    global stop_animation
    stop_animation = True

# Callback function for restart button
def on_restart_button_clicked(b):
    on_stop_button_clicked(b)
    MC.c_B = float(C_B_slider.value) / 100
    MC.initialize_lattice()
    updater_plot(MC.lattice, [], [], [])
    step_info.value = "Lattice restarted"
    play_button.disabled = False

# Create interactive widgets with layout adjustments
layout = widgets.Layout(width='300px')
style = {'description_width': 'initial'}
temperature_input = widgets.Text(value='700', description='Temperature (K):', layout=layout, style=style)
total_steps_input = widgets.Text(value=f'{int(1e6)}', description='Total Steps:', layout=layout, style=style)
interval_input = widgets.Text(value='500', description='Display Interval:', layout=layout, style=style)
E_AA_input = widgets.Text(value='-1.0', description='E_AA (eV):', layout=layout, style=style)
E_BB_input = widgets.Text(value='-1.0', description='E_BB (eV):', layout=layout, style=style)
E_AB_input = widgets.Text(value='-0.9', description='E_AB (eV):', layout=layout, style=style)
C_B_slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Concentration B (%):', 
                               layout=widgets.Layout(width='900px'),style=style)
play_button = widgets.Button(description="Play", layout=layout)
stop_button = widgets.Button(description="Stop", layout=layout)
restart_button = widgets.Button(description="Restart", layout=layout)
step_info = widgets.Label(value=f"Lattice initialized.", layout=widgets.Layout(width='1000px'))

play_button.on_click(lambda b: asyncio.ensure_future(on_play_button_clicked(b)))
stop_button.on_click(on_stop_button_clicked)
restart_button.on_click(on_restart_button_clicked)

# Arrange widgets in a more organized layout
inputs_box = widgets.VBox([
    widgets.HBox([temperature_input,total_steps_input,interval_input]),
    widgets.HBox([E_AA_input, E_BB_input, E_AB_input]),
    widgets.HBox([C_B_slider]),
    widgets.HBox([play_button, stop_button,restart_button]),
    step_info
    ])


# Set up Plotly figure
L = 32 # lattice size
MC = CanonicalMonteCarloSimulation(L)
initialize_plot(MC.lattice)
# Display widgets and figure
display(fig,inputs_box)
